In [1]:
import sys
# use line-buffering for both stdout and stderr
# sys.stdout = open(sys.stdout.fileno(), mode='w', buffering=1)
# sys.stderr = open(sys.stderr.fileno(), mode='w', buffering=1)

import hydra
from omegaconf import OmegaConf
import pathlib
from equi_diffpo.workspace.base_workspace import BaseWorkspace
import hydra
from omegaconf import OmegaConf
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
 
 
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
from equi_diffpo.workspace.base_workspace import BaseWorkspace
from equi_diffpo.policy.diffusion_unet_hybrid_image_policy import DiffusionUnetHybridImagePolicy
from equi_diffpo.dataset.base_dataset import BaseImageDataset
from equi_diffpo.env_runner.base_image_runner import BaseImageRunner
from equi_diffpo.common.checkpoint_util import TopKCheckpointManager
from equi_diffpo.common.json_logger import JsonLogger
from equi_diffpo.common.pytorch_util import dict_apply, optimizer_to
from equi_diffpo.model.diffusion.ema_model import EMAModel
from equi_diffpo.model.common.lr_scheduler import get_scheduler

from equi_diffpo.dataset.robomimic_replay_image_dataset import RobomimicReplayImageDataset 
from equi_diffpo.env_runner.robomimic_image_runner import RobomimicImageRunner
from datetime import datetime

from tqdm import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Sampler 
import json

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/ns1254/miniconda3/envs/equidiff/lib/python3.9/site-packages/robomimic/scripts/setup_macros.py
)


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/ns1254/miniconda3/envs/equidiff/lib/python3.9/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
now = f"{datetime.now():%Y_%m_%d_%H_%M}"
now

'2025_01_07_01_43'

In [3]:
max_steps = {
    'stack_d1': 400,
    'stack_three_d1': 400,
    'square_d2': 1200,
    'threading_d2': 400,
    'coffee_d2': 1200,
    'three_piece_assembly_d2': 500,
    'hammer_cleanup_d1': 500,
    'mug_cleanup_d1': 1200,
    'kitchen_d1': 800,
    'nut_assembly_d0': 500,
    'pick_place_d0': 1000,
    'coffee_preparation_d1': 800,
    'tool_hang': 700,
    'can': 400,
    'lift': 400,
    'square': 400,
}

def get_ws_x_center(task_name):
    if task_name.startswith('kitchen_') or task_name.startswith('hammer_cleanup_'):
        return -0.2
    else:
        return 0.

def get_ws_y_center(task_name):
    return 0.

OmegaConf.register_new_resolver("get_max_steps", lambda x: max_steps[x], replace=True)
OmegaConf.register_new_resolver("get_ws_x_center", get_ws_x_center, replace=True)
OmegaConf.register_new_resolver("get_ws_y_center", get_ws_y_center, replace=True)

# allows arbitrary python code execution in configs using the ${eval:''} resolver
OmegaConf.register_new_resolver("eval", eval, replace=True)

In [4]:
# python train.py --config-name=train_diffusion_unet task_name=square_d2 
#            n_demo=100 dataset_path=/home/ns1254/dataset_mimicgen/square134_2_0ind_abs.hdf5 
#            dataset_filter_key="g40b30"

In [5]:
with open('/home/ns1254/gib/segs/segs_square_g40f10s10.txt', 'r') as f:
    data = json.load(f)

segs_todo= data['segs_todo']
dataset_path = data['dataset_path']
dataset_filter_key = data['dataset_filter_key']
fn_seg = data['fn_seg']
data = data['data']
segs_toremove=json.loads(data)

print(f"segs_todo: {segs_todo}")
print(f"dataset_path: {dataset_path}")
print(f"dataset_filter_key: {dataset_filter_key}")
print(f"fn_seg: {fn_seg}") 
print(f"data: {data}")

segs_todo: remove
dataset_path: /home/ns1254/dataset_mimicgen/square134_2_0ind_abs.hdf5
dataset_filter_key: g40f10s10
fn_seg: /home/ns1254/dataset_mimicgen/square_bad_segs_0ind.txt
data: {"demo_14": [[0, 621]], "demo_21": [[0, 612]], "demo_23": [[0, 598]], "demo_24": [[0, 997]], "demo_39": [[0, 634]], "demo_42": [[0, 846]], "demo_4": [[0, 654]], "demo_49": [[0, 517]], "demo_50": [[0, 713]], "demo_58": [[0, 527]], "demo_10": [[200, 488]], "demo_12": [[164, 602]], "demo_19": [[177, 979]], "demo_20": [[162, 673]], "demo_26": [[211, 662]], "demo_28": [[218, 1120]], "demo_2": [[199, 730]], "demo_31": [[166, 753]], "demo_33": [[191, 968]], "demo_34": [[171, 1134]]}


In [6]:
list( segs_toremove.keys() )[:3]

['demo_14', 'demo_21', 'demo_23']

In [5]:
dataset_path = "/home/ns1254/dataset_mimicgen/square134_2_0ind_abs.hdf5" 
# dataset_filter_key = "g40b30"
config_path='equi_diffpo/config' 
config_name = 'train_diffusion_unet.yaml'
task_name='square_d2' 
n_demo = 100

In [6]:
with initialize(version_base=None, config_path=config_path):
    cfg_org = compose(
        config_name=config_name,
        overrides=[ 
            f"task_name={task_name}",
            f"n_demo={n_demo}"
        ],
    )
    print(cfg_org)
    
OmegaConf.resolve(cfg_org) 

{'name': 'diff_c', '_target_': 'equi_diffpo.workspace.train_diffusion_unet_hybrid_workspace.TrainDiffusionUnetHybridWorkspace', 'shape_meta': '${task.shape_meta}', 'exp_name': 'default', 'task_name': 'square_d2', 'n_demo': 100, 'horizon': 16, 'n_obs_steps': 2, 'n_action_steps': 8, 'n_latency_steps': 0, 'dataset_obs_steps': '${n_obs_steps}', 'past_action_visible': False, 'obs_as_global_cond': True, 'dataset': 'equi_diffpo.dataset.robomimic_replay_image_dataset.RobomimicReplayImageDataset', 'dataset_path': 'data/robomimic/datasets/${task_name}/${task_name}_abs.hdf5', 'dataset_filter_key': 'all', 'policy': {'_target_': 'equi_diffpo.policy.diffusion_unet_hybrid_image_policy.DiffusionUnetHybridImagePolicy', 'shape_meta': '${shape_meta}', 'noise_scheduler': {'_target_': 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler', 'num_train_timesteps': 100, 'beta_start': 0.0001, 'beta_end': 0.02, 'beta_schedule': 'squaredcos_cap_v2', 'variance_type': 'fixed_small', 'clip_sample': True, 'prediction_

In [7]:
cls = hydra.utils.get_class(cfg_org._target_)
cls

equi_diffpo.workspace.train_diffusion_unet_hybrid_workspace.TrainDiffusionUnetHybridWorkspace

In [9]:
cfg_org.keys()

dict_keys(['name', '_target_', 'shape_meta', 'exp_name', 'task_name', 'n_demo', 'horizon', 'n_obs_steps', 'n_action_steps', 'n_latency_steps', 'dataset_obs_steps', 'past_action_visible', 'obs_as_global_cond', 'dataset', 'dataset_path', 'dataset_filter_key', 'policy', 'ema', 'dataloader', 'val_dataloader', 'optimizer', 'training', 'logging', 'checkpoint', 'multi_run', 'task'])

In [10]:
class TrainDiffusionUnetHybridWorkspace(BaseWorkspace):
    include_keys = ['global_step', 'epoch']

    def __init__(self, cfg: OmegaConf, output_dir=None):
        super().__init__(cfg, output_dir=output_dir)

        # set seed
        seed = cfg.training.seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # configure model
        self.model: DiffusionUnetHybridImagePolicy = hydra.utils.instantiate(cfg.policy)

        self.ema_model: DiffusionUnetHybridImagePolicy = None
        if cfg.training.use_ema:
            self.ema_model = copy.deepcopy(self.model)

        # configure training state
        self.optimizer = hydra.utils.instantiate(
            cfg.optimizer, params=self.model.parameters())

        # configure training state
        self.global_step = 0
        self.epoch = 0

In [11]:
output_dir = f"/home/ns1254/equidiff/outputs/square_{now}"
# os.mkdir(output_dir)
os.makedirs(output_dir, exist_ok=True)
workspace = TrainDiffusionUnetHybridWorkspace(cfg_org, output_dir=output_dir)

self = workspace


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_gripper_qpos', 'robot0_eef_pos', 'robot0_eef_quat']
using obs modality: rgb with keys: ['agentview_image', 'robot0_eye_in_hand_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/home/ns1254/miniconda3/envs/equidiff/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ns1254/miniconda3/envs/equidiff/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Diffusion params: 2.556120e+08
Vision params: 2.239418e+07


In [12]:
cfg = copy.deepcopy(self.cfg)

In [13]:
cfg.task.dataset.items()

dict_items([('_target_', 'equi_diffpo.dataset.robomimic_replay_image_dataset.RobomimicReplayImageDataset'), ('n_demo', 100), ('shape_meta', {'obs': {'agentview_image': {'shape': [3, 84, 84], 'type': 'rgb'}, 'robot0_eye_in_hand_image': {'shape': [3, 84, 84], 'type': 'rgb'}, 'robot0_eef_pos': {'shape': [3]}, 'robot0_eef_quat': {'shape': [4]}, 'robot0_gripper_qpos': {'shape': [2]}}, 'action': {'shape': [10]}}), ('dataset_path', 'data/robomimic/datasets/square_d2/square_d2_abs.hdf5'), ('dataset_filter_key', 'all'), ('horizon', 16), ('pad_before', 1), ('pad_after', 7), ('n_obs_steps', 2), ('abs_action', True), ('rotation_rep', 'rotation_6d'), ('use_legacy_normalizer', False), ('use_cache', True), ('seed', 42), ('val_ratio', 0.02)])

In [14]:
# resume training
# if cfg.training.resume:
#     lastest_ckpt_path = self.get_checkpoint_path()
#     if lastest_ckpt_path.is_file():
#         print(f"Resuming from checkpoint {lastest_ckpt_path}")
#         self.load_checkpoint(path=lastest_ckpt_path)
  
new_config = {key: value for key, value in cfg.task.dataset.items() if key != '_target_'}
new_config['dataset_path'] = dataset_path
new_config['dataset_filter_key'] = dataset_filter_key

del new_config["n_demo"]


obs_shape_meta_config = {key: value for key, value in new_config['shape_meta']['obs'].items()}
obs_shape_meta_config['demo_no'] = {'shape': [], 'type': 'low_dim'}
obs_shape_meta_config['index_in_demo'] = {'shape': [], 'type': 'low_dim'}

new_config['shape_meta']['obs'] = obs_shape_meta_config


dataset = RobomimicReplayImageDataset(**new_config )

----------------------------
dataset_path:  /home/ns1254/dataset_mimicgen/square134_2_0ind_abs.hdf5
dataset filter key:  g40f10s10
----------------------------
------------------------------ self.n_demo=60------------------------------
Acquiring lock on cache.
Loading cached ReplayBuffer from Disk.
Loaded!


In [15]:
obs_shape_meta_config

{'agentview_image': {'shape': [3, 84, 84], 'type': 'rgb'},
 'robot0_eye_in_hand_image': {'shape': [3, 84, 84], 'type': 'rgb'},
 'robot0_eef_pos': {'shape': [3]},
 'robot0_eef_quat': {'shape': [4]},
 'robot0_gripper_qpos': {'shape': [2]},
 'demo_no': {'shape': [], 'type': 'low_dim'},
 'index_in_demo': {'shape': [], 'type': 'low_dim'}}

In [16]:
for key in new_config['shape_meta']['obs']:
    print(key, new_config['shape_meta']['obs'][key])

agentview_image {'shape': [3, 84, 84], 'type': 'rgb'}
robot0_eye_in_hand_image {'shape': [3, 84, 84], 'type': 'rgb'}
robot0_eef_pos {'shape': [3]}
robot0_eef_quat {'shape': [4]}
robot0_gripper_qpos {'shape': [2]}
demo_no {'shape': [], 'type': 'low_dim'}
index_in_demo {'shape': [], 'type': 'low_dim'}


In [17]:
# data = dataset.sampler.sample_sequence(index)
# data.keys()

In [18]:
index=10
data = dataset.__getitem__(index)
data.keys()

dict_keys(['obs', 'action'])

In [19]:
data['obs'].keys()

dict_keys(['agentview_image', 'robot0_eye_in_hand_image', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos', 'demo_no', 'index_in_demo'])

In [20]:
data['obs']['agentview_image'].shape

torch.Size([2, 3, 84, 84])

In [21]:
def parse_1_data(data):
    """ 
    data: at time t from dataset.
    #each timestamp can contain multiple uid because of obs_horizon
    """
    if 'demo_no' not in data['obs']:
        raise Exception("Please add demo_no and index_in_demo to the obs first.")
    
    demo_nos = data['obs']['demo_no']
    indices_in_demo = data['obs']['index_in_demo']
    return demo_nos, indices_in_demo

In [22]:
# segs_toremove={
#  'demo_0': [ (5,20), (30, 40) ],
#  'demo_1': [ (12, 20) ],
#  'demo_12': [(0, 10), (15, 20), (30,33) ],
# }

In [23]:
remove_ids={}
for key in segs_toremove.keys():
    segs = segs_toremove[key]
    ids = [] 
    for start, end in segs:
        ids.extend(range(start, end + 1))  # Include the end value
    remove_ids[key]=ids

In [24]:
valid_indices =[]  #in the dataset.

for index in tqdm( range(len(dataset)) ):
    data = dataset.__getitem__(index)
    demo_no, indices_in_demo = parse_1_data(data) 

    assert torch.all( demo_no[0]==demo_no[1] )                 #obs history from same demo
    demo_name=f'demo_{int(demo_no[0])}'
    ids = indices_in_demo.numpy().astype(int)
    
    should_remove = False
    if demo_name in remove_ids:
        should_remove = bool(set(remove_ids[demo_name]) & set(ids))
    if should_remove: continue 
    valid_indices.append(index)

len(valid_indices), len(dataset)

100%|██████████| 60207/60207 [08:07<00:00, 123.51it/s]


(47278, 60207)

In [25]:
demo_name, indices_in_demo.numpy().astype(int)

('demo_99', array([346, 347]))

In [25]:
# dataset.lowdim_keys.remove('demo_no')
# dataset.lowdim_keys.remove('index_in_demo')  # remove the added helper keys (demo_no and index in demo)

In [27]:
dataset.lowdim_keys 

['robot0_eef_pos',
 'robot0_eef_quat',
 'robot0_gripper_qpos',
 'demo_no',
 'index_in_demo']

In [44]:
class CustomIndicesSampler(Sampler):
    def __init__(self, custom_indices):
        self.custom_indices = np.random.permutation(custom_indices)

    def __iter__(self): 
        return iter(self.custom_indices)

    def __len__(self):
        return len( self.custom_indices )

In [45]:
sampler = CustomIndicesSampler(valid_indices)

In [46]:
cfg.dataloader

{'batch_size': 64, 'num_workers': 4, 'shuffle': True, 'pin_memory': True, 'persistent_workers': True}

In [47]:
new_config = {key:value for key,value in cfg.dataloader.items()}
new_config['shuffle'] = False
new_config['sampler'] = sampler

In [48]:
new_config

{'batch_size': 64,
 'num_workers': 4,
 'shuffle': False,
 'pin_memory': True,
 'persistent_workers': True,
 'sampler': <__main__.CustomIndicesSampler at 0x7fba3473b430>}

### Note: train and validation need to be handled for partial trajs

In [49]:
train_dataloader = DataLoader(dataset, **new_config)
# 
# normalizer = dataset.get_normalizer()

# configure validation dataset
val_dataset = dataset.get_validation_dataset()
val_dataloader = DataLoader(val_dataset, **cfg.val_dataloader)

In [50]:
dataset.train_mask

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [51]:
for batch in tqdm(train_dataloader, desc="checking progress", unit="batch"):    
    obs=batch['obs']
    demo_no=batch['obs']['demo_no']
    index_in_demo=batch['obs']['index_in_demo']

    assert torch.all( demo_no[:,0]==demo_no[:,1] )                    #obs history from same demo
    demo_names=[f'demo_{int(i)}' for i in demo_no[:,0]]

    for demo_name, index in zip(demo_names, index_in_demo):
        i,j= int( index[0].item() ), int( index[1].item() )
        # print(demo_name, i, j)
        if demo_name in segs_toremove:
            for s,e in segs_toremove[demo_name]: 
                if s<=i<=e or s<=j<=e:                          #i and j within start-end
                    print(f"should remove {demo_name} {i} {j}")
                    break


    # break 

checking progress: 100%|██████████| 739/739 [01:33<00:00,  7.87batch/s]


In [52]:
len(train_dataloader)

739

In [53]:
len( train_dataloader.sampler )

47278

In [54]:
len(val_dataloader)

15

In [55]:
for batch in tqdm(val_dataloader, desc="checking progress", unit="batch"):
    obs=batch['obs']
    demo_no=batch['obs']['demo_no']
    index_in_demo=batch['obs']['index_in_demo']

    assert torch.all( demo_no[:,0]==demo_no[:,1] )                    #obs history from same demo
    demo_names=[f'demo_{int(i)}' for i in demo_no[:,0]]

    for demo_name, index in zip(demo_names, index_in_demo):
        i,j= int( index[0].item() ), int( index[1].item() )
        # print(demo_name, i, j)
        if demo_name in segs_toremove:
            for s,e in segs_toremove[demo_name]: 
                if s<=i<=e or s<=j<=e:                          #i and j within start-end
                    print(f"should remove {demo_name} {i} {j}")
                    break


    # break 

checking progress: 100%|██████████| 15/15 [00:18<00:00,  1.26s/batch]
